# k means ||

In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("prova iniziale")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/28 16:14:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=prova iniziale>

# Import data from sklearn



In [4]:
!pip install scikit-learn # to be run at the launch of "docker compose up" 

In [5]:
import numpy as np
import pandas as pd
import random
import sklearn.datasets #va installato

from pyspark.sql.functions import col
from pyspark.sql.functions import mean
from pyspark.sql.functions import stddev
from pyspark.ml.feature import StandardScaler

In [6]:
# Hyperparameters
k = 20
G = 3 # Giosu factor
len_df = 5_000

In [7]:
# Import dataframe directly from sklearn
prova =  sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True)
X = prova.data

# prova = sc.parallelize(sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True))

In [8]:
# Show the dataframe (to be deleted for a bigger dataframe) (TO BE DELETED)
X

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,b'tcp',b'http',b'SF',310,1881,0,0,0,0,...,86,255,1.0,0.0,0.01,0.05,0.0,0.01,0.0,0.0
494017,0,b'tcp',b'http',b'SF',282,2286,0,0,0,0,...,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0
494018,0,b'tcp',b'http',b'SF',203,1200,0,0,0,0,...,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0
494019,0,b'tcp',b'http',b'SF',291,1200,0,0,0,0,...,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0


# Create a Spark dataframe
And then acquire important informations about the dataframe: # columns, # rows, # partitions, the schema, ...

In [9]:
# Create the spark dataframe with a smaller chunk of data (just to work easily)
X_smaller = X.iloc[random.sample(range(1, len(X.index)), len_df)]
spark_X = spark.createDataFrame(X_smaller)

/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
# Lock the spark dataframe
#spark_X.persist() # non serve?

In [11]:
# Check the number of partitions the DataFrame is divided into the three workers yet
spark_X.rdd.getNumPartitions()

3

In [12]:
# Define the number of rows and columns in the dataframe
n_rows = spark_X.count()

23/08/28 16:15:13 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


TO CHANGE THE COLUMN/ROWS NUMBER EVALUATION

In [13]:
# Delete the non-numerical values in an easy way
# clean_X = spark_X.select('*').drop('protocol_type', 'service', 'flag')
# clean_X.printSchema()

# Delete the non-numerical values in a reusable way
col_type = np.array(spark_X.dtypes)
types = col_type[:,1] 
colnames = col_type[:,0]
clean_X = spark_X.select([col(colnames[i]) for i in range(len(colnames)) if not types[i] == 'binary'])
# clean_X.printSchema()

In [14]:
# Normalize the numerical columns

i = 0
means = np.zeros(len(clean_X.columns))
devstd = np.zeros(len(clean_X.columns))
# Calculate mean and variance for each column
for column in clean_X.columns:
    means[i] = float(clean_X.select(mean(column)).head()[0])
    devstd[i] = float(clean_X.select(stddev(column)).head()[0])
    i = i+1

# norm_X = clean_X
means, devstd

(array([4.8560800e+01, 2.9155388e+03, 6.1800340e+02, 0.0000000e+00,
        6.2000000e-03, 0.0000000e+00, 3.8000000e-02, 0.0000000e+00,
        1.5620000e-01, 5.1200000e-02, 4.0000000e-04, 4.0000000e-04,
        5.8400000e-02, 6.0000000e-04, 0.0000000e+00, 1.6000000e-03,
        0.0000000e+00, 0.0000000e+00, 1.6000000e-03, 3.2670980e+02,
        2.8755100e+02, 1.7701800e-01, 1.7704800e-01, 5.7366000e-02,
        5.7536000e-02, 7.9015400e-01, 2.1118000e-02, 2.7848000e-02,
        2.3011800e+02, 1.8738360e+02, 7.5230600e-01, 3.1110000e-02,
        5.9443400e-01, 7.4280000e-03, 1.7728000e-01, 1.7664200e-01,
        5.8448000e-02, 5.7242000e-02]),
 array([6.28753959e+02, 1.02730574e+05, 4.66518236e+03, 0.00000000e+00,
        1.34778311e-01, 0.00000000e+00, 8.34203320e-01, 0.00000000e+00,
        3.63081156e-01, 3.36631029e+00, 1.99979995e-02, 2.82842712e-02,
        3.79524310e+00, 3.16202462e-02, 0.00000000e+00, 4.89685571e-02,
        0.00000000e+00, 0.00000000e+00, 3.99719846e-02, 2.14

TO BE NORMALIZED!!

# Step 0: Useful functions

In [15]:
# Define the function for the distance between the cluster centers and the pandas dataframe
def distance(xrow,broadC):
    '''Distance between a dataframe.row
    and the broadcasted list of centers
    in the form of dataframe.row:
    
    broadC = sc.broadcast(center_rows)
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)
    centers = broadC.value
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.min(the_ds)

def find_closest_center(xrow,broadC):
    '''Distance between a dataframe.row
    and the broadcasted list of centers
    in the form of dataframe.row:
    
    broadC = sc.broadcast(center_rows)
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)
    centers = broadC.value
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.argmin(the_ds)

# Define the function for the logarithm of the cost phi
def phi(df,centers ):
    '''logarithm of the cost phi'''
    return np.log( df.rdd.map(lambda row: distance(row,centers)).reduce(lambda x,y: x + y) )

# Is this reasonable?? --> maybe we should cast it to int ?
def evaluate_l(log_phi, k, G):
    return G * k/log_phi # G = over-oversampling factor

# Function to select a row based on its probability
def select_row(x):
    if x > np.random.uniform(low = 0, high = 1):
        return True
    else:
        return False

### This is NOT $\Phi$. This is $log(\Phi)$. Pay attention budeo.

# Step 1: Choose a random sample from the dataset
This is the required step to begin the algorithm (doesn't need to be parallelized, since it is a select task)

In [16]:
# Choose the first sample randomly: select the random row
random_n = [np.random.randint(0, n_rows)]

# Define a function that is able to select the single row in the dataframe
def getrows(df, rownums=None): # copied from the internet
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])
random_sample = getrows(clean_X, random_n).collect()


In [17]:
# Broadcasting the row over the workers
bCent = sc.broadcast(random_sample)

# To show the content of the broadcast:
# b.value

# Step 2: Initial cost
Then we evaluate the cost function (sum of the squares after the first selection)

In [18]:
# Initial cost
initial_cost = phi(clean_X, bCent)

## Step 2.5: evaluate number of iterations

After having evaluated $\log \phi$, we find a number of iterations which is of the same order of magnitude.

In [19]:
int(initial_cost)

31

In [20]:
n_iter = int(initial_cost)

# Step 3: For loop
Implement the for loop in order to evaluate probabilities and choose new centers



In [21]:
# Loop over the log(phi) steps
# for i in range(int(initial_cost))
phi_iter = initial_cost
l = evaluate_l(phi_iter, k, G)

#Add initial probabilities
#probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)

#clean_X_p = clean_X.withColumn('probabilities', col(probabilities)) 

#distance = (clean_X.select(columns)
 #                      .withColumn('initial_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames))
  #             )

In [22]:
l

1.8987691847675352

In [23]:
clean_X.count()

5000

In [24]:
import time

In [25]:
%%time

i = 0
while i < n_iter or len(bCent.value) < k:
    '''
    Nel ciclo for:
        - Evaluate for each row l * d()^2 / phi
        - Sample with that probability
        - Broadcast centers to nodes
        - Evaluate new cost
        
    Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
    '''

    # Evaluate the probability and select the new rows
    #probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)
    new_rows = clean_X.rdd\
    .filter(lambda row: select_row(np.exp(np.log(distance(row,bCent))-phi_iter)*l)).collect()
    
    # Update the broadcast
    bCent = sc.broadcast(bCent.value + new_rows)
    
    # Evaluate new cost
    phi_iter =  phi(clean_X, bCent)
    
    i += 1
    print("\n i: ",len(bCent.value), " \n")


 i:  3  




 i:  6  




 i:  8  




 i:  13  




 i:  14  



...


 i:  57  




 i:  57  




 i:  60  




 i:  60  




 i:  62  

CPU times: user 1.24 s, sys: 214 ms, total: 1.45 s
Wall time: 2min 29s


In [26]:
phi_iter

18.795553482610504

In [27]:
len(bCent.value)

62

# Step 4: Select a subset of the possible centroids using k-means ++

Using kmeans ++:

Algorithm 1 k-means++(k) initialization.
1: C ← sample a point uniformly at random from X
2: while |C| < k do
2
3:
Sample x ∈ X with probability dφX(x,C)
(C)
4:
C ← C ∪ {

In [28]:
bCent_list = bCent.value

In [29]:
dfCent = spark.createDataFrame(bCent_list)

In [30]:
over_sampled_centers = dfCent.count()

In [31]:
bCent.unpersist()

In [32]:
dfCent.rdd.getNumPartitions()

3

In [33]:
bCent = sc.broadcast(bCent_list)

In [34]:
len(bCent.value)

62

## Reduce size of dfCent to have only `k` centroids 

#### Step 1: calc weights $w_x$
set wx to be the number of points in X closer to x than any other point in C

In [35]:
from pyspark.sql.functions import count

In [36]:
wx = clean_X.rdd.map(lambda row: (find_closest_center(row,bCent),1)).reduceByKey(lambda x,y: x+y).takeOrdered(over_sampled_centers) #o lambda x,y: x+y

In [37]:
wx = (np.array(wx)[:,1]).astype(float)

In [38]:
wx /= np.sum(wx)
wx

array([1.078e-01, 2.000e-04, 2.000e-04, 8.000e-04, 1.200e-03, 2.800e-03,
       2.000e-04, 3.400e-03, 4.494e-01, 2.000e-04, 2.000e-04, 4.200e-03,
       1.200e-03, 1.600e-03, 1.200e-03, 6.000e-04, 8.000e-04, 2.000e-04,
       1.100e-02, 1.148e-01, 8.000e-04, 1.800e-03, 2.000e-04, 6.000e-04,
       2.000e-04, 1.220e-02, 2.000e-04, 3.800e-03, 2.000e-04, 1.800e-03,
       2.000e-04, 4.400e-03, 6.000e-04, 4.000e-04, 2.000e-04, 2.000e-04,
       2.400e-03, 1.200e-03, 2.400e-03, 1.480e-02, 6.800e-03, 8.000e-04,
       1.000e-03, 6.000e-04, 4.000e-04, 3.800e-03, 3.440e-02, 8.000e-03,
       1.400e-03, 1.144e-01, 2.400e-03, 7.000e-03, 2.100e-02, 1.000e-03,
       4.000e-04, 6.200e-03, 1.800e-03, 4.000e-04, 2.000e-04, 4.000e-03,
       1.140e-02, 2.200e-02])

#### Step 2: Use weighted `k-means++`
    2.1 Draw 1 random center
    2.2 update cost function
    2.3 repeat 1-2 until happy

In [39]:
first_center = np.random.choice(a=range(wx.shape[0]), size=1, p=wx) 
#first point sampled uniformly wrt distance
first_center

array([49])

In [40]:
bCent_ultimate = sc.broadcast(getrows(dfCent,first_center).collect())
bCent_ultimate.value

[Row(duration=0, src_bytes=0, dst_bytes=0, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, logged_in=0, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=249, srv_count=3, serror_rate=1.0, srv_serror_rate=1.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=0.01, diff_srv_rate=0.07, srv_diff_host_rate=0.0, dst_host_count=255, dst_host_srv_count=3, dst_host_same_srv_rate=0.01, dst_host_diff_srv_rate=0.08, dst_host_same_src_port_rate=0.0, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=1.0, dst_host_srv_serror_rate=1.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0)]

In [41]:
phi0 = phi(dfCent, bCent_ultimate)
phi0

31.59822672715055

In [43]:
type(bCent_ultimate.value)

list

In [44]:
%%time

i = 0
while i < k: #n_iter or len(bCent.value) < k:
    '''
    Nel ciclo for:
        - Evaluate for each row l * d()^2 / phi
        - Sample with that probability
        - Broadcast centers to nodes
        - Evaluate new cost
        
    Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
    '''
    
    # Evaluate the probability and select the new rows
    #probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)
    rows_prob = np.array(dfCent.rdd\
    .map(lambda row: (np.exp(np.log(distance(row,bCent_ultimate))-phi0))).collect())
    
    #sample new random center
    if i == 0:
        print('rows_prob@wx \n',rows_prob@wx, '\n')
    
    another_index = np.random.choice(a=range(wx.shape[0]), size=1, p = rows_prob*wx/(rows_prob@wx) ) 
    
    print(i ,'step: another_index = ', another_index,'\n')
    
    another_center = getrows(dfCent, another_index).collect()
    
    # Update the broadcast
    bCent_ultimate = sc.broadcast(bCent_ultimate.value + another_center)
    
    # Evaluate new cost
    phi0 = phi(dfCent, bCent_ultimate)
    
    i += 1
    print("\n i: ",i,"; bC.val =",bCent_ultimate.value, " \n")

rows_prob@wx 
 0.00020029381640820657 

0 step: another_index =  [1] 

1 step: another_index =  [6] 

2 step: another_index =  [5] 

3 step: another_index =  [13] 

4 step: another_index =  [50] 

5 step: another_index =  [55] 

6 step: another_index =  [20] 

7 step: another_index =  [22] 

8 step: another_index =  [8] 

9 step: another_index =  [26] 

10 step: another_index =  [34] 

11 step: another_index =  [15] 

12 step: another_index =  [25] 

13 step: another_index =  [53] 

14 step: another_index =  [58] 

15 step: another_index =  [17] 

16 step: another_index =  [30] 

17 step: another_index =  [45] 

18 step: another_index =  [38] 

19 step: another_index =  [59] 

CPU times: user 997 ms, sys: 229 ms, total: 1.23 s
Wall time: 13.5 s


In [45]:
sum(rows_prob*wx)

0.0004990531228289555

In [46]:
'''
- Prendere la lista dei centroidi e farla diventare un distributed dataset
- Fare una NUOVA lista di centroidi 
- for i in range(k):
    pesca un singolo dato con probabilità d(x)/phi --> come fare?
    aggiungilo alla lista

'''

'\n- Prendere la lista dei centroidi e farla diventare un distributed dataset\n- Fare una NUOVA lista di centroidi \n- for i in range(k):\n    pesca un singolo dato con probabilità d(x)/phi --> come fare?\n    aggiungilo alla lista\n\n'

# TO DO:
    - Capire come scegliere l
    - Fare la normalizzazione dei dati (se la facciamo)
    - Capire se abbiamo ottimizzato nel modo giusto
    - Implementare tutto k means (optional)
    - Creare un file gemello con un dataframe visualizzabile e vedere se ha senso
    - Iniziare a fare i benchmark su cloud veneto
    - Grafici etc...

# Stop worker and master
Stop the running Spark context (sc) and Spark session (spark)

In [47]:
# sc.stop()
# spark.stop()

Finally, use `docker compose down` to stop and clear all running containers.